In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import nn_ops

In [10]:
truths = tf.Variable( [[0, 1, 2, 0, 1]])
end = tf.one_hot(truths, 3, 1, 0)
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(end))

[[[1 0 0]
  [0 1 0]
  [0 0 1]
  [1 0 0]
  [0 1 0]]]


In [4]:
a = np.arange(0, 60, 1,dtype=np.float32)# start at 0 count up by 2, stop before 30
a = a.reshape(3,4,5)
# b = np.ones([1,5])
b = np.array([[1,1,1,1],[-1,-1,-1,-1],[0,1,0,0]])
a = tf.Variable(a , dtype = tf.float32 )
b = tf.Variable(b , dtype = tf.float32 )
c = tf.tile(tf.expand_dims(b, axis=-1), [1, 1, 8])
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    e = sess.run([c])

In [2]:
def generate_skeleton_representation(in_value, indicate_matrix):
    # in_value: [batch_size, seq_length, feature_dim]
    # indicate_matrix: [batchsize, seq_length]
    batch_size = tf.shape(in_value)[0]
    # seq_length = tf.shape(in_value)[1]
    feature_dim = in_value.get_shape().as_list()[-1]
    indicate_matrix = tf.tile(tf.expand_dims(indicate_matrix, axis=-1), [1, 1, feature_dim])

    # 1: type -1:target 0:other
    indicate_matrix_type = tf.maximum(indicate_matrix, tf.zeros_like(indicate_matrix, dtype=tf.float32))
    indicate_matrix_target = -tf.minimum(indicate_matrix, tf.zeros_like(indicate_matrix, dtype=tf.float32))

    type_repr = tf.reduce_sum(tf.multiply(in_value, indicate_matrix_type), axis=1)
    target_repr = tf.reduce_sum(tf.multiply(in_value, indicate_matrix_target), axis=1)
    type_repr = tf.reshape(type_repr, shape=[batch_size, feature_dim])
    target_repr = tf.reshape(target_repr, shape=[batch_size, feature_dim])

    w_type = tf.get_variable("fuse_w1", shape=[feature_dim, feature_dim], dtype=tf.float32)
    w_target = tf.get_variable("fuse_w2", shape=[feature_dim, feature_dim], dtype=tf.float32)
    b = tf.get_variable("fuse_b", shape=[feature_dim], dtype=tf.float32)
    return nn_ops.bias_add(tf.matmul(type_repr, w_type) + tf.matmul(target_repr, w_target), b)

In [7]:
e[0].shape

(3, 4, 8)

In [6]:
help(tf.one_hot)
    

Help on function one_hot in module tensorflow.python.ops.array_ops:

one_hot(indices, depth, on_value=None, off_value=None, axis=None, dtype=None, name=None)
    Returns a one-hot tensor.
    
    The locations represented by indices in `indices` take value `on_value`,
    while all other locations take value `off_value`.
    
    `on_value` and `off_value` must have matching data types. If `dtype` is also
    provided, they must be the same data type as specified by `dtype`.
    
    If `on_value` is not provided, it will default to the value `1` with type
    `dtype`
    
    If `off_value` is not provided, it will default to the value `0` with type
    `dtype`
    
    If the input `indices` is rank `N`, the output will have rank `N+1`. The
    new axis is created at dimension `axis` (default: the new axis is appended
    at the end).
    
    If `indices` is a scalar the output shape will be a vector of length `depth`
    
    If `indices` is a vector of length `features`, the outp

In [66]:
import os
import pandas as pd

def unique_list(list_):
    assert isinstance(list_, list)
    if list_ != []:
        return list(set(list_))
    return []

def read_file(file_path):
    """
    read train & test data from original excel file
    :param file_path: original excel file path, str
    :return samples: list of train & test samples in the form of dict, list[dict]
    :return id2sample: dict of index to sample, dict
    """
    print('--')
    root_dir, file = os.path.split(file_path)
    data_frame = pd.read_excel(file_path)
    samples = [data_frame.iloc[index].to_dict() for index in range(len(data_frame))]
    print(len(samples))
    num = 0
#     temp = []
#     for sample in samples:
#         if judge_have_qa_pair(temp, sample):
#             a=1
#         else:
#             num+=1
#             temp.append(sample)
#     return temp
#     print('---')
    processed_samples = []
    for sample in samples:
        if str(sample['label']) == '1' and len(have_same_qa_pair(samples, sample)) >= 2 and len(set(have_same_qa_pair(samples,sample)))==1 :
            sample.update({'target':'all'})
            processed_samples.append(sample)
        if str(sample['label']) == '1' and len(have_same_qa_pair(samples, sample)) >= 2 and '0' in have_same_qa_pair(samples,sample):
            processed_samples.append(sample)
        if str(sample['label']) == '2'and len(have_same_qa_pair(samples, sample)) >= 2 and len(set(have_same_qa_pair(samples,sample)))==1 :
            sample.update({'target':'none'})
            processed_samples.append(sample)
        if len(have_same_qa_pair(samples, sample)) == 1:
            print(sample)
            processed_samples.append(sample)
    print(len(processed_samples))
    processed_samples_new = []
    for sample in processed_samples:
        if judge_have_qa_pair(processed_samples_new, sample) :
            a = 1
        else:
            processed_samples_new.append(sample)        
    print(len(processed_samples_new))
    return processed_samples_new
file_path = r'D:\yl\desktop\temp_2.xlsx'
samples = read_file(file_path)

--
12923
{'question': '您会选择挑战性比较大，或是比较平淡的工作', 'target': '平淡', 'answer': '挑战性比较大的', 'label': 0}
{'question': '当您的工作规划和公司的长远规划发生冲突时，您会选择妥协还是跳槽或是重新规划自己的工作生涯', 'target': '跳槽', 'answer': '干不下去就换', 'label': 1}
6667
5034


In [64]:
len(samples)

6667

In [67]:
with open(r'D:\yl\desktop\temp.txt','w', encoding='utf-8') as outfile:
    for sample in samples:
        outfile.writelines('\t'.join([sample['question'], sample['answer'], sample['target'],'\n']))

In [21]:
import os
import pandas as pd
import random

def unique_list(list_):
    assert isinstance(list_, list)
    if list_ != []:
        return list(set(list_))
    return []

def read_file(file_path):
    """
    read train & test data from original excel file
    :param file_path: original excel file path, str
    :return samples: list of train & test samples in the form of dict, list[dict]
    :return id2sample: dict of index to sample, dict
    """
    print('--')
    root_dir, file = os.path.split(file_path)
    data_frame = pd.read_excel(file_path)
    samples = [data_frame.iloc[index].to_dict() for index in range(len(data_frame))]
    alternative_answer = '我再想想，我再考虑考虑，不记得了，不想回答，不想回答这个问题，没法回答你，这个问题不好回答，别问我，我不清楚，\
    我不知道，这个问题有点难弄，我不明白你的意思，没在意过这个问题，没留意这个问题，我不关心，给个提示，我不想选，这个问题有点难，你想问啥，\
    你想知道啥，我怎么可能知道，这个问题你应该问别人，不太懂，看情况，具体情况具体分析，看到底是什么吧，不一定，不知道，不理解，不明白，\
    不了解，看心情，不好说，不确定，不记得了，你问别人吧，看看别人是怎么选的'.split('，')
    outfile = open(r'D:\yl\desktop\temp.csv','w')
    print(alternative_answer)
    passed_questions = []
    for sample in samples:        
        if str(sample['问题类别']) == '1':
            if sample['问题'] not in passed_questions:
                for index in random.sample(range(len(alternative_answer)),4):
                    index = int(index)
                    answer = alternative_answer[index]
                    outfile.writelines('\t'.join([sample['所属领域'], str(sample['问题类别']), sample['问题'], answer,'none_1','\n']))
            passed_questions.append(sample['问题']) 
    outfile.close()
file_path = r'D:\yl\desktop\data_all.xlsx'
samples = read_file(file_path)

--
['我再想想', '我再考虑考虑', '不记得了', '不想回答', '不想回答这个问题', '没法回答你', '这个问题不好回答', '别问我', '我不清楚', '    我不知道', '这个问题有点难弄', '我不明白你的意思', '没在意过这个问题', '没留意这个问题', '我不关心', '给个提示', '我不想选', '这个问题有点难', '你想问啥', '    你想知道啥', '我怎么可能知道', '这个问题你应该问别人', '不太懂', '看情况', '具体情况具体分析', '看到底是什么吧', '不一定', '不知道', '不理解', '不明白', '    不了解', '看心情', '不好说', '不确定', '不记得了', '你问别人吧', '看看别人是怎么选的']


In [10]:
a = random.sample(range(5),4)
type(a)

list

In [53]:
# def have_same_qa_pair(samples, cur_sample):
#     num = 0
#     for sample in samples:
#         if sample['question'] == cur_sample['question'] and sample['answer'] == cur_sample['answer']:
#             num += 1
#     return num

def have_same_qa_pair(samples, cur_sample):
    temp = []
    for sample in samples:
        if sample['question'] == cur_sample['question'] and sample['answer'] == cur_sample['answer']:
            temp.append(str(sample['label']))
    return temp

def judge_have_qa_pair(samples, cur_sample):
    for sample in samples:
        if sample['question'] == cur_sample['question'] and sample['answer'] == cur_sample['answer']:
            return True
    return False

In [ ]:
@ run